# Collection of further information

This notebook performs a domain analysis based on the data `real_domains.csv`. First, it checks the correctness of the specified `A records` and `MX records`. The current A-records and MX-records of the top-level domain are determined. In the next step, the `redirects` and `HTTP status codes` are obtained from the top-level domains. The Python package called `dnspython` is used for the determination. In addition, a further analysis of the MX records determines their locations and the corresponding ASNs. Free IP geolocation databases from MaxMind are used for this.

## Setup

First, the required installations and imports are made, the session is initialized and the basis data is collected from the Postgres database:

In [1]:
# Required installations
!pip install ipynb
!pip install dnspython
!pip install geoip2

# Required imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from ipynb.fs.full.Functions import *

# PostgreSQL access data
host = "bda_gr4_database"
port = "5432"
database = "domainanalysis"
user = "postgres"
password = "postgres"

# PostgreSQL connection url
connection = f"jdbc:postgresql://{host}:{port}/{database}"

# Create a Spark session
spark = SparkSession.builder \
    .appName("domain_analysis") \
    .getOrCreate()

# Read data from the database
domains_df = spark.read \
                .format("jdbc") \
                .option("url", connection) \
                .option("dbtable", "domain") \
                .option("user", user) \
                .option("password", password) \
                .load()

# Display the data frame
#domains_df.limit(15).toPandas()

     |████████████████████████████████| 241 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 286 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 315 kB 12.5 MB/s eta 0:00:01
  Created wheel for maxminddb: filename=maxminddb-2.0.3-py2.py3-none-any.whl size=15284 sha256=9b357b65529db826177450dd3edcf970a449bc796171233be170a1fe83e88945
  Stored in directory: /home/jovyan/.cache/pip/wheels/ad/0c/82/79e41ebe67044c68a5087459a9ea7447cb0756019071318197
Successfully built maxminddb


21/07/05 22:55:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/07/05 22:55:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/07/05 22:55:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## 1. Validation of A and MX records

In the following chapter, the functions are imported from the notebook `Functions.ipynb` in order to use them for the definition of the `UDFs`. In doing so, the current `a-records` and `mx-records` are to be determined from the `top-level-domain`.  If the records cannot be determined, the returned errors are stored in separate columns:

In [2]:
args = domains_df.select("top_level_domain").toPandas().values.reshape(-1)

In [3]:
result_map_a_record = execute_threaded_fn(getARecords, args)
result_map_a_record_error = execute_threaded_fn(getARecords_error, args)
result_map_mx_record = execute_threaded_fn(getMXRecords, args)
result_map_mx_record_error = execute_threaded_fn(getMXRecords_error, args)

In [4]:
def getARecords_udf(top_level_domain):
    try:
        return result_map_a_record[top_level_domain]
    except KeyError:
        return getARecords(top_level_domain)

def getARecords_error_udf(top_level_domain):
    return result_map_a_record_error[top_level_domain]

def getMXRecords_udf(top_level_domain):
    return result_map_mx_record[top_level_domain]

def getMXRecords_error_udf(top_level_domain):
    return result_map_mx_record_error[top_level_domain]

In [5]:
# Creating of UDF's
udf_getARecords = udf(getARecords_udf, ArrayType(StringType()))
udf_getARecords_error = udf(getARecords_error_udf,  IntegerType())

udf_getMXRecords = udf(getMXRecords_udf, ArrayType(StringType()))
udf_getMXRecords_error = udf(getMXRecords_error_udf,  IntegerType())

udf_remove_last_char_in_array = udf(fn_remove_dot, ArrayType(StringType()))

Next, the A and MX records are checked by calling the UDFs and creating the returned results in new columns:

In [6]:
# Create the new columns with the results
domains_checked_df = domains_df.withColumn("a_record_checked", udf_getARecords("top_level_domain")) \
                            .withColumn("a_record_checked_error", udf_getARecords_error("top_level_domain")) \
                            .withColumn("mx_record_checked", udf_getMXRecords("top_level_domain")) \
                            .withColumn("mx_record_checked", udf_remove_last_char_in_array(col("mx_record_checked"))) \
                            .withColumn("mx_record_checked_error", udf_getMXRecords_error("top_level_domain"))
domains_checked_df.limit(15).toPandas()

21/07/05 22:57:19 WARN DAGScheduler: Broadcasting large task binary with size 1130.4 KiB


,top_level_domain,mx_record,a_record,a_record_checked,a_record_checked_error,mx_record_checked,mx_record_checked_error
0,1-billig-reisen.de,"[mx2e48.netcup.net, mail.1-billig-reisen.de]",[91.204.46.70],[91.204.46.70],0,"[mail.1-billig-reisen.de, mx2e48.netcup.net]",0
1,06249.de,[localhost],[91.195.241.137],[64.190.62.111],0,[localhost],0
2,123-search.de,None,[52.58.78.16],"[75.2.26.18, 99.83.153.108]",0,[],0
3,1558.de,[localhost],[91.195.241.137],[64.190.62.111],0,[localhost],0
4,00zins.de,None,None,None,1,None,1
5,123-neu.de,[localhost],[91.195.241.137],None,1,None,1
6,1-rad-laden.de,"[mx2.securemail.name, mx1.securemail.name]",[45.67.71.51],[45.67.71.51],0,"[mx2.securemail.name, mx1.securemail.name]",0
7,0815kredit.de,[localhost],[91.195.241.137],[64.190.62.111],0,[localhost],0
8,01026telecom.de,[mail6.comnet.nl],[77.72.174.30],[77.72.174.30],0,[mail6.comnet.nl],0
9,10719.de,[localhost],[91.195.241.137],[64.190.62.111],0,[localhost],0


In preparation for writing to the database, the data frame is put into the correct order:

In [7]:
# Changing the order of the data frame
domains_checked_df = domains_checked_df.select("top_level_domain", "a_record", "a_record_checked", "a_record_checked_error", "mx_record", "mx_record_checked", "mx_record_checked_error")

Last but not least, let's check if the records have either not been changed, changed completely or partially in comparison to the original data.

In [8]:
# Count the occurrence of checked A-records
a_record_checked_count = domains_checked_df.withColumn('a_record_checked', explode(col('a_record_checked'))) \
        .groupBy('a_record_checked') \
        .count()

# Count the occurrence of checked MX-records
mx_record_checked_count = domains_checked_df.withColumn('mx_record_checked', explode(col('mx_record_checked'))) \
        .groupBy('mx_record_checked') \
        .count()

# Finally, create new data frames only containing the top 10 A-/MX-records
a_record_count_top_ten_df = a_record_checked_count.orderBy(['count'], ascending = [False]).limit(10)
mx_record_count_top_ten_df = mx_record_checked_count.orderBy(['count'], ascending = [False]).limit(10)

Finally, the columns `a_record` and `mx_record` are removed from the `domains_checked_df` data frame. Furthermore, the first 15 rows of the data frame are displayed as a check for writing to the database:

In [9]:
# Remove columns and show the data frame
domains_checked_df = domains_checked_df.drop("a_record").drop("mx_record")
domains_checked_df.limit(15).toPandas()

21/07/05 22:57:20 WARN DAGScheduler: Broadcasting large task binary with size 1125.0 KiB


,top_level_domain,a_record_checked,a_record_checked_error,mx_record_checked,mx_record_checked_error
0,1-billig-reisen.de,[91.204.46.70],0,"[mail.1-billig-reisen.de, mx2e48.netcup.net]",0
1,06249.de,[64.190.62.111],0,[localhost],0
2,123-search.de,"[75.2.26.18, 99.83.153.108]",0,[],0
3,1558.de,[64.190.62.111],0,[localhost],0
4,00zins.de,None,1,None,1
5,123-neu.de,None,1,None,1
6,1-rad-laden.de,[45.67.71.51],0,"[mx2.securemail.name, mx1.securemail.name]",0
7,0815kredit.de,[64.190.62.111],0,[localhost],0
8,01026telecom.de,[77.72.174.30],0,[mail6.comnet.nl],0
9,10719.de,[64.190.62.111],0,[localhost],0


In [10]:
# Display the data frame
a_record_count_top_ten_df.limit(10).toPandas()

,a_record_checked,count
0,64.190.62.111,2221
1,52.58.78.16,224
2,80.150.6.143,148
3,185.230.63.107,87
4,185.230.63.186,87
5,185.230.63.171,87
6,81.169.145.162,41
7,81.20.137.85,41
8,88.99.113.60,41
9,81.169.145.86,40


In [11]:
# Display the data frame
mx_record_count_top_ten_df.limit(10).toPandas()

,mx_record_checked,count
0,localhost,2222
1,smtpin.rzone.de,601
2,mx01.kundenserver.de,390
3,mx00.kundenserver.de,388
4,mx00.ionos.de,223
5,mx01.ionos.de,223
6,smtp-02.tld.t-online.de,148
7,smtp-01.tld.t-online.de,148
8,mx2.jimdo.com,134
9,mx1.jimdo.com,134


After the generated data frames have been checked, they must be written to the PostgreSQL database for data visualisation. To enable a faster write speed, `repartition` and `batchsize` are specified:

In [12]:
# Write data frames to the PostgreSQL database
a_record_count_top_ten_df.write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "a_record_checked_count_global") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

mx_record_count_top_ten_df.write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "mx_record_checked_count_global") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

domains_checked_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_records_checked") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

21/07/05 22:57:22 WARN DAGScheduler: Broadcasting large task binary with size 1128.1 KiB
21/07/05 22:57:23 WARN DAGScheduler: Broadcasting large task binary with size 1128.5 KiB


## 2. Determination of redirects and HTTP status codes

This chapter contains the definition of the `redirects` and `HTTP status` codes to the top level domains. In the following, the functions are imported from the function notebook `Functions.ipynb` in order to use them for the definition of the `UDFs`:

In [13]:
result_map_redirect = execute_threaded_fn(getRedirectUrl, args)
result_map_statuscode = execute_threaded_fn(getStatusCodeUrl, args)

In [14]:
def getRedirectUrl_udf(top_level_domain):
    return result_map_redirect[top_level_domain]

def getStatusCodeUrl_udf(top_level_domain):
    return result_map_statuscode[top_level_domain]

In [15]:
# Creating of UDF's
udf_getRedirectUrl = udf(getRedirectUrl_udf, StringType())
udf_getStatusCodeUrl = udf(getStatusCodeUrl_udf, IntegerType())

In this block, the `redirections` and the `HTTP status codes` are determined by calling the UDF functions. For this purpose, the column `top_level_domain` is passed as a parameter. Subsequently, the redirections and status codes are stored in new columns:

In [16]:
# Create the new columns with the results
domains_redirect_df = domains_df.withColumn("redirection", udf_getRedirectUrl("top_level_domain")) \
                                .withColumn("status_code", udf_getStatusCodeUrl("top_level_domain"))
domains_redirect_df.limit(15).toPandas()

,top_level_domain,mx_record,a_record,redirection,status_code
0,1-billig-reisen.de,"[mx2e48.netcup.net, mail.1-billig-reisen.de]",[91.204.46.70],1-billig-reisen.de,200.0
1,06249.de,[localhost],[91.195.241.137],06249.de,403.0
2,123-search.de,None,[52.58.78.16],godaddy.com:443,200.0
3,1558.de,[localhost],[91.195.241.137],1558.de,403.0
4,00zins.de,None,None,None,NaN
5,123-neu.de,[localhost],[91.195.241.137],None,NaN
6,1-rad-laden.de,"[mx2.securemail.name, mx1.securemail.name]",[45.67.71.51],1-rad-laden.de,403.0
7,0815kredit.de,[localhost],[91.195.241.137],0815kredit.de,403.0
8,01026telecom.de,[mail6.comnet.nl],[77.72.174.30],01026telecom.de,200.0
9,10719.de,[localhost],[91.195.241.137],10719.de,403.0


For storage in the PostgreSQL database, the `a_record` and `mx_record` columns are removed from the `domains_redirect_df` data frame and the first 15 rows of the data frame are displayed for checking for writing to the database:

In [17]:
# Remove columns and show the data frame
domains_redirect_df = domains_redirect_df.drop("a_record").drop("mx_record")
domains_redirect_df.limit(15).toPandas()

,top_level_domain,redirection,status_code
0,1-billig-reisen.de,1-billig-reisen.de,200.0
1,06249.de,06249.de,403.0
2,123-search.de,godaddy.com:443,200.0
3,1558.de,1558.de,403.0
4,00zins.de,None,NaN
5,123-neu.de,None,NaN
6,1-rad-laden.de,1-rad-laden.de,403.0
7,0815kredit.de,0815kredit.de,403.0
8,01026telecom.de,01026telecom.de,200.0
9,10719.de,10719.de,403.0


In [18]:
domains_redirect_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_redirection") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

## 3. MX record location and ASN determination

In this section, the `locations` and `providers` are identified based on the ANS using the MX records already checked. For each MX record the iso code, location, postal code, latitude, longitude and the organisation of the autonomous system are presented:

In [19]:
# Read data from the database
domain_records_checked_df = spark.read \
                .format("jdbc") \
                .option("url", connection) \
                .option("dbtable", "domain_records_checked") \
                .option("user", user) \
                .option("password", password) \
                .load()

# Drop columns
domain_records_checked_df = domain_records_checked_df.drop("a_record_checked").drop("a_record_checked_error").drop("mx_record_checked_error")

# Display the data frame
domain_records_checked_df.limit(20).toPandas()

,top_level_domain,mx_record_checked
0,100zufuss.de,None
1,111weitewege.de,"[mx00.kundenserver.de, mx01.kundenserver.de]"
2,089booker.de,[089booker.de]
3,18-jahre.de,[localhost]
4,112shop.de,[w01a5318.kasserver.com]
5,1a-finanzvergleich.de,None
6,100000000000.de,[localhost]
7,170grad.de,"[mx1.jimdo.com, mx2.jimdo.com]"
8,123newsletter.de,[localhost]
9,123-kostenlos.de,[localhost]


Next, the functions from the function notebook `Functions.ipynb` that are to be used for the definition of the `UDFs` are imported. After that, StructTypes of StructFields are created to define the return types:

In [20]:
schema_location = StructType([
    StructField("iso_code", StringType(), True),
    StructField("city", StringType(), True),
    StructField("postal", StringType(), True),
    StructField("latitude", StringType(), True),
    StructField("longitude", StringType(), True)])

schema_asn = StructType([
    StructField("autonomous_system_organization", StringType(), True)])

udf_getGeoLite2_Location = udf(getGeoLite2_Location, schema_location)
udf_getGeoLite2_ASN = udf(getGeoLite2_ASN, schema_asn)

In order to be able to perform the location and provider query, the ip address of the mx records must first be found out. Based on this, the `locations` and the `providers` are determined and the data frame `domains_mx_record_geolite2_df` is generated:

In [21]:
# Create the new columns with the results
domains_mx_record_geolite2_df = domain_records_checked_df.select(domain_records_checked_df.top_level_domain,explode(domain_records_checked_df.mx_record_checked).alias('mx_record_checked'))
domains_mx_record_geolite2_df = domains_mx_record_geolite2_df.withColumn("mx_record_ip", udf_getARecords("mx_record_checked")) \
                            .withColumn('mx_record_ip', explode(col('mx_record_ip'))) \
                            .withColumn("location", udf_getGeoLite2_Location("mx_record_ip")) \
                            .withColumn("asn", udf_getGeoLite2_ASN("mx_record_ip")) \
                            .select("top_level_domain", "mx_record_checked", "mx_record_ip", "location.*", "asn.*")

For storage in the PostgreSQL database, the first 15 rows of the data frame are displayed for checking for writing to the database:

In [22]:
# Display the data frame
#domains_mx_record_geolite2_df.limit(15).toPandas()

In [23]:
# Write the data frame to the PostgreSQL database
domains_mx_record_geolite2_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_mx_record_geolite2") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

## 4. IPv6 and SOA information fetch

This section contains the fetch of IPv6, nameserver information and a ranking of the top ten master nameservers and the companies behind.
The following block generates the required functions as user defined function.

In [24]:
result_map_aaaa = execute_threaded_fn(IPv6Record, args)
result_map_aaaa_error = execute_threaded_fn(IPv6Record_error, args)

In [25]:
def getAAAA_record_udf(top_level_domain):
    return result_map_aaaa[top_level_domain]

def getAAAA_record_error_udf(top_level_domain):
    return result_map_aaaa_error[top_level_domain]

In [26]:
udf_IPv6Record = udf(getAAAA_record_udf, BooleanType())
udf_IPv6Record_error = udf(getAAAA_record_error_udf, IntegerType())

### IPv6 information
The IPv6 information provided by the following code are the availability of IPv6 (`ipv6_available`, boolean) and the possible error code during a request (`ipv6_error`). Therefore, the dataframe 'domains_ipv6_df' is created.

In [27]:
# Create df for ipv6 data
domains_ipv6_df = domains_df.withColumn("ipv6_available", udf_IPv6Record('top_level_domain'))\
                        .withColumn("ipv6_error", udf_IPv6Record_error("top_level_domain"))\
                        .drop('mx_record').drop('a_record')

domains_ipv6_df.limit(10).toPandas()

,top_level_domain,ipv6_available,ipv6_error
0,1-billig-reisen.de,True,0
1,06249.de,False,2
2,123-search.de,False,2
3,1558.de,False,2
4,00zins.de,None,1
5,123-neu.de,None,1
6,1-rad-laden.de,False,2
7,0815kredit.de,False,2
8,01026telecom.de,False,2
9,10719.de,False,2


In [28]:
#Write the data frame to the PostgreSQL database
domains_ipv6_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "ip_v6_information") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

### SOA information
Among the SOA record, the names of the nameservers `nameservers` (iincluding master server) are fetched in the following. The variable `nameservers_error` indicates if problems ocurred during the request wheres as `nameservers_count` contains the number of nameservers used per domain.

In [29]:
result_map_soa_information = execute_threaded_fn(getSOAInformation, args)
result_map_soa_information_error = execute_threaded_fn(getSOAInformation_error, args)
result_map_nameservers = execute_threaded_fn(getNameServers, args)
result_map_nameservers_error = execute_threaded_fn(getNameServers_error, args)

In [30]:
def getSOAInformation_udf(top_level_domain):
    return result_map_soa_information[top_level_domain]

def getSOAInformation_error_udf(top_level_domain):
    return result_map_soa_information_error[top_level_domain]

def getNameServers_udf(top_level_domain):
    return result_map_nameservers[top_level_domain]

def getNameServers_error_udf(top_level_domain):
    return result_map_nameservers_error[top_level_domain]

In [31]:
udf_getSOAInformation = udf(getSOAInformation_udf, ArrayType(StringType()))
udf_getSOAInformation_error = udf(getSOAInformation_error_udf, IntegerType())
udf_getNameServers = udf(getNameServers_udf, ArrayType(StringType()))
udf_getNameServers_error = udf(getNameServers_error_udf, IntegerType())

In [32]:
# Create df for SOA data and drop unnecessary columns
domains_soa_df = domains_df.withColumn("soa_information", udf_getSOAInformation("top_level_domain"))\
                        .withColumn("soa_information_error", udf_getSOAInformation_error("top_level_domain"))\
                        .withColumn("nameservers", udf_getNameServers("top_level_domain"))\
                        .withColumn("nameservers_error", udf_getNameServers_error("top_level_domain"))\
                        .drop('mx_record').drop('a_record')

# Add nameserver count variable
def count_arr(arr): return 0 if arr == None else len(arr)
count_arr_udf = udf(count_arr, IntegerType())
domains_soa_df = domains_soa_df.withColumn("nameservers_count", count_arr_udf('nameservers'))
domains_soa_df.limit(10).toPandas()

21/07/05 23:12:45 WARN DAGScheduler: Broadcasting large task binary with size 1936.2 KiB


,top_level_domain,soa_information,soa_information_error,nameservers,nameservers_error,nameservers_count
0,1-billig-reisen.de,[root-dns.netcup.net. dnsadmin.netcup.net. 202...,0,"[second-dns.netcup.net., third-dns.netcup.net....",0,3
1,06249.de,[ns1.sedoparking.com. hostmaster.sedo.de. 2018...,0,"[ns2.sedoparking.com., ns1.sedoparking.com.]",0,2
2,123-search.de,[ns5.afternic.com. dns.jomax.net. 2021022300 2...,0,"[ns6.afternic.com., ns5.afternic.com.]",0,2
3,1558.de,[ns1.sedoparking.com. hostmaster.sedo.de. 2018...,0,"[ns1.sedoparking.com., ns2.sedoparking.com.]",0,2
4,00zins.de,None,2,None,1,0
5,123-neu.de,None,2,None,1,0
6,1-rad-laden.de,[ns1-tec.de. webmaster.1-rad-laden.de. 2021062...,0,"[ns1-tec.de., ns2-tec.de.]",0,2
7,0815kredit.de,[ns1.sedoparking.com. hostmaster.sedo.de. 2018...,0,"[ns1.sedoparking.com., ns2.sedoparking.com.]",0,2
8,01026telecom.de,[ns1.comnet.nl. hostmaster.01026telecom.de. 20...,0,"[ns1.comnet.nl., ns3.comnet.nl.]",0,2
9,10719.de,[ns1.sedoparking.com. hostmaster.sedo.de. 2018...,0,"[ns2.sedoparking.com., ns1.sedoparking.com.]",0,2


The details concerning the master nameserver are provided as ArrayType. As this is inconvenient with regard to the data structure (atomicity), the column `soa_infos_rep` is converted into a string type to separate its contents into separate variables.
These are the name of the primary master nameserver `soa_name`, the refresh time of the SOA record executed by the secondary nameservers (`refresh`), the 'Time to live' (TTL), which defines how long a secondary nameserver caches the requested SOA information `minimum`.

In [33]:
# Change ArrayType<String> into String as preparation for information separation
domains_soa_df = domains_soa_df.withColumn("soa_infos_rep", concat_ws(" ", "soa_information"))

# Split SOA information into separate columns (all String)
split_col = split(domains_soa_df['soa_infos_rep'], ' ')
domains_soa_df = domains_soa_df.withColumn('soa_name', split_col.getItem(0))\
                        .withColumn('refresh', split_col.getItem(3))\
                        .withColumn('minimum', split_col.getItem(6))

# Helping function to catch empty entries concerning the master nameserver
def replace_empty_strings(x):
    return when(col(x) == "", None).otherwise(col(x))

domains_soa_df = domains_soa_df.withColumn("soa_name", replace_empty_strings("soa_name"))

Some fetched entries contain unnecessary characters which need to be removed (dot at the end of the name).

In [34]:
# Remove last dot per soa mname
domains_soa_df = domains_soa_df.withColumn('soa_name', regexp_replace('soa_name', '.$', ''))   

# Remove last dot per nameserver entry
lambda_dot_remove = lambda arr: [x[:-1] for x in arr]
def fn_remove_dot(arr): return None if arr == None else lambda_dot_remove(arr)
udf_remove_last_char_in_array = udf(fn_remove_dot, ArrayType(StringType()))

domains_soa_df = domains_soa_df \
    .select("*") \
    .withColumn('nameservers', udf_remove_last_char_in_array(col('nameservers')))

In [35]:
# Change data type of the time setting columns into int
domains_soa_df = domains_soa_df.withColumn("refresh", domains_soa_df["refresh"].cast(IntegerType()))\
                        .withColumn("minimum", domains_soa_df["minimum"].cast(IntegerType()))\
                        .drop('soa_information').drop('soa_infos_rep')

In [36]:
#Write the data frame to the PostgreSQL database
domains_soa_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "soa") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

21/07/05 23:12:45 WARN DAGScheduler: Broadcasting large task binary with size 1945.9 KiB
21/07/05 23:12:45 WARN DAGScheduler: Broadcasting large task binary with size 1942.0 KiB


### Top ten master namservers and related company information
This section creates a dataframe concerning the top ten master neameservers and the companies behind.

In [37]:
# Count the occrence of SOA records
soa_name_count_top_ten_df = domains_soa_df.withColumn('soa_name', (col('soa_name'))) \
        .groupBy('soa_name') \
        .count()

soa_name_count_top_ten_df = soa_name_count_top_ten_df.orderBy(['count'], ascending = [False]).limit(10)
soa_name_count_top_ten_df.limit(10).toPandas()

,soa_name,count
0,ns1.sedoparking.com,2221
1,None,790
2,ns5.kasserver.com,570
3,root-dns.netcup.net,200
4,dns01-tld.t-online.de,150
5,ns1.ns-serve.net,141
6,ns1.jimdo.com,137
7,ns1.undeveloped.com,133
8,ns81.domaincontrol.com,132
9,ns.inwx.de,131


The company informations per master nameserver will be added by a request of its IP adresses `ipv4`. The IP address(es) are the used within the functions concerning GeoLite2 to get the company information ()

In [38]:
# Remove None entries (copmany information not available)
soa_name_count_top_ten_df = soa_name_count_top_ten_df.na.drop(subset=["soa_name"])

soa_name_count_top_ten_df = soa_name_count_top_ten_df.withColumn("ipv4", udf_getARecords("soa_name"))\
                      .withColumn('ipv4', explode(col('ipv4'))) \
                      .withColumn("location", udf_getGeoLite2_Location("ipv4")) \
                      .withColumn("asn", udf_getGeoLite2_ASN("ipv4")) \
                      .select("soa_name", "count", "ipv4", "location.*", "asn.*")
#soa_name_count_top_ten_df.limit(5).toPandas()

In [39]:
#Write the data frame to the PostgreSQL database
soa_name_count_top_ten_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "soa_top_ten") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()

21/07/05 23:12:46 WARN DAGScheduler: Broadcasting large task binary with size 1275.4 KiB
21/07/05 23:12:47 WARN DAGScheduler: Broadcasting large task binary with size 1263.6 KiB
